### In this notebook we obtain the normal form of the benchmarking circuits

In [1]:
import numpy as np
import pytket as tk
from pytket import passes as tkp
from pytket import circuit as tkc
from pytket.circuit.display import render_circuit_jupyter as print_circ
from pytket.qasm import circuit_from_qasm_str, circuit_to_qasm,circuit_from_qasm
import zipfile
import os
import shutil

The documentation has through descriptions of the functions and their purpose. This notebook goes through the procedure for obtaining the normal form, <br>
which we apply to all the circuits in the benchmarking folder, "nam-circuits" in the original Quantinuum challenge repository

In [2]:
def adding_registers(reading_file, modified_file):
    '''
    Counts the number of H gates h_count, adds a flag at
    each hadamard, and creates a quantum register and classical
    register of size h_count.
    '''
    h_count = 0
    h_qbit_list = []
    with open(reading_file, 'r') as file, open('temp.qasm', 'w') as new_file:
        for line in file:
            if line.strip().startswith('h q['):
                h_count += 1
                start_idx = line.find('[')
                end_idx = line.find(']')
                h_qbit_list.append(int(line[start_idx+1:end_idx]))
                new_file.write('//spotted\n')
            new_file.write(line)
    file.close()
    new_file.close()

    with open('temp.qasm', 'r') as file, open(modified_file, 'w') as new_file:
        for line in file:
            if line.strip().startswith('qreg'):
                new_file.write(line)
                new_file.write(f'qreg a[{h_count}];\n')
                new_file.write(f'creg ma[{h_count}];\n')
            else:
                new_file.write(line)


    return h_qbit_list

def hadamard_string(qbit_number, ancilla_number):
    '''
    Creates the two qubit blocks that
    substitute the hadamard gates
    '''

    return          f'h a[{ancilla_number}];\n' \
                    +   f's q[{qbit_number}];\n' \
                    +   f's a[{ancilla_number}];\n' \
                    +   f'cx q[{qbit_number}], a[{ancilla_number}];\n' \
                    +   f'sdg a[{ancilla_number}];\n' \
                    +   f'cx a[{ancilla_number}], q[{qbit_number}];\n' \
                    +   f'cx q[{qbit_number}], a[{ancilla_number}];\n' \
                    +   f'h a[{ancilla_number}];\n' \
                    +   f'measure a[{ancilla_number}] -> ma[{ancilla_number}];\n' \
                    +   f'if (ma[{ancilla_number}] == 1) x q[{qbit_number}];\n'

def add_h_blocks(old_file, modified_file, h_qbit_list):
    '''
    Substitutes the hadamard gates
    by the two qubit blocks defined in the paper
    '''
    flag = False
    ancilla_idx = 0

    with open(old_file, 'r') as file, open(modified_file, 'w') as new_file:
        for line in file:
            if flag:
                flag = False
                continue
            else:
                pass

            if line.strip().startswith('//spotted'):
                flag = True
                new_file.write(line)
                new_file.write(hadamard_string(h_qbit_list[ancilla_idx], ancilla_idx))
                ancilla_idx += 1
            else:
                new_file.write(line)



First we have to apply the notebook correcting_qasm_order over the circuit files to obtain them in the Clifford basis and <br>
in the proper order. From that, we obtain qasm_files.zip, which we have to unzip. 

In [12]:
# Extract the files of qasm_files.zip (quantum circuits in Clifford + T gateset) to a folder
extract_to = 'qasm_files'
zip_path = 'circuits/qasm_files.zip'
os.makedirs(extract_to)
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    # Extract all the contents into the directory
    zip_ref.extractall(extract_to)
    print(f"Files have been extracted to {extract_to}")

Files have been extracted to qasm_files


Now we gadgetize all the circuits in the folder

In [13]:
#Function for gadgetizing all the qasm files
def gadgetize_qasm_files(source_dir, target_dir):
    os.makedirs(target_dir, exist_ok=True)
    for filename in os.listdir(source_dir):
        if filename.endswith('.qasm'):
            reading_file = os.path.join(source_dir, filename)
            temp_file = os.path.join(target_dir, 'temp_' + filename)
            modified_file = os.path.join(target_dir, filename)
            h_qbit_list = adding_registers(reading_file, temp_file)
            add_h_blocks(temp_file, modified_file, h_qbit_list)
            os.remove(temp_file)

source_dir = 'qasm_files/'
target_dir = 'qasm_gadgetized_files/'
# Process all .qasm files in the source directory
gadgetize_qasm_files(source_dir, target_dir)

Now we check if the gadgetization worked, testing on one of the circuits in the folder

In [7]:
qc = circuit_from_qasm('qasm_files/barenco_tof_3_modified.qasm')
print_circ(qc)

qc_gadgetized = circuit_from_qasm('qasm_gadgetized_files/barenco_tof_3_modified.qasm')
print_circ(qc_gadgetized)


In [ ]:
def return_Uf(filename, flag):
    with open(filename, 'r') as f:
        lines = f.readlines()
    for i in range(len(lines)-1, -1, -1):
        if flag in lines[i]:
            break
    if i == 0:
        return None, None
    else:
        del lines[i]
        ancilla = lines[i+1]
        del lines[i+1]
        with open(filename, 'w') as f:
            f.writelines(lines)
        ancilla = ancilla[ancilla.find("c["):ancilla.find("]") + 1]
        return lines[i+1:], ancilla

In [ ]:
filename = 'modified_adder_8.qasm'
flag = 'spotted'
while True:
    Uf, measure = return_Uf(filename, flag)
    if Uf == None:
        break
    print(Uf)
